In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.timeseries import TimeSeries
import numpy as np
import pandas as pd
from pandas import DataFrame as df, Timestamp
import csv
import requests
import os
import time
#import module for mysql 
from os import path
import mysql.connector as mysql
from pandas import DataFrame

def randomize () :
    global zkey
    global _API_KEY_TEST
    zkey += 1
    maxcounter = len(names) - 1
    if zkey > maxcounter :
        zkey = 0
    _API_KEY_TEST = names[zkey]
    print ("the API key right now is : "+_API_KEY_TEST)
    return _API_KEY_TEST

#function for making TTM data
def makettm(tabtemp,x):
    tabtemp = DataFrame(tabtemp)
    width = x
    shifted = tabtemp.shift(0)
    window = shifted.rolling(window=width)
    test = window.sum()
    complete = test.fillna(value=0)
    return complete

#function for add rows to fundamental rows to be equal with timeseries rows and fill nan
def addrow(source,target,ori_table):
    data=[]
    #y=pd.DataFrame()
    counter=0
    for i in range(0,10):
        if source[i]==target[0]:
            break
        counter+=1
    for j in range(0,counter):
        data.insert(j, {np.NaN,np.NaN,np.NaN})
    y=pd.concat([pd.DataFrame(data), ori_table], ignore_index=True) #repeat for all variable
    y.fillna(method='bfill',inplace=True)
    return [y,counter]

def connect(db_name):
    try:
        return mysql.connect(
            host='localhost',
            user='root',
            password='minyak23',
            database=db_name)
    except Error as e:
        print(e)
def step_1() :
    global step 
    step = "step_1"
    data1, meta_data = fd.get_income_statement_quarterly(entity)  #get income statement quarterly data
    data1.to_csv(r'C:\Varians\Income Statement Quarterly.csv', index=None, header=True)
    print("Succsesfully Pull Data 1 from web")
def step_2() :  
    global step 
    step = "step_2"
    data2, meta_data = fd.get_cash_flow_quarterly(entity) #get cash flow quarterly data
    data2.to_csv(r'C:\Varians\Cash Flow Quarterly.csv', index=None, header=True)
    print("Succsesfully Pull Data 2 from web")
def step_3():
    global step 
    step = "step_3"
    data3, meta_data = fd.get_balance_sheet_quarterly(entity) #get balance sheet quarterly data
    data3.to_csv(r'C:\Varians\Balance Sheet Quarterly.csv', index=None, header=True)
    print("Succsesfully Pull Data 3 from web")
def step_4():
    global step 
    step = "step_4"
    global OV
    fd = FundamentalData(key=_API_KEY_TEST, output_format='json')
    OV, meta_data = fd.get_company_overview( entity)
    split_OV=OV
    #write tuple/list overview to csv file
    with open(r'C:\Varians\Overview2.csv', 'w') as g:
        writer1=csv.writer(g,lineterminator='\n')
        writer1.writerow(split_OV.keys())
        writer1.writerow(split_OV.values())
        
    print("Succsesfully Pull Data Overview from web")
        
def step_5():
    global step 
    step = "step_5"
    ts = TimeSeries(key=_API_KEY_TEST, output_format='pandas')
    last_price, meta_data = ts.get_monthly(entity)
    last_price.to_csv(r'C:\Varians\Last Price.csv', index=True, header=True)
    print("Succsesfully Pull Data Timeseries from web")

if __name__ == "__main__":        
    # define the name of the directory to be created
    path = "/Varians"
    #creating folder on C:
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s" % path)

    with open(r'C:\Varians\stocklist.csv') as slist:
        stocklist = pd.read_csv(slist, header = None)

    stocklist = stocklist.rename(columns={0 : 'company', 1 : 'market', 2 : 'ticker'})
    stocklist.ticker = stocklist.ticker.str.strip()

    _API_KEY_TEST1 = 'G23MAAVKWB5TMPMV'
    _API_KEY_TEST2 = '6PE06AMZAM1MCGFX' #myanuarfirdaus
    _API_KEY_TEST3 = '6HZYCBP71FGOR1O5' #myanuarfirdaus23
    _API_KEY_TEST4 = 'X5LNMI2AESR1YQCU' #muh_yanuar_firdaus@yahoo.com
    _API_KEY_TEST5 = 'YBBKWRK4VSTP4GZH' #anggiengineer@yahoo.com
    _API_KEY_TEST6 = 'X3Z3LQ31Z9B1OYX9' #myanuarfirdaus@hotmail.com
    names = [_API_KEY_TEST1,_API_KEY_TEST2,_API_KEY_TEST3,_API_KEY_TEST4,_API_KEY_TEST5,_API_KEY_TEST6]
    # names = [_API_KEY_TEST4,_API_KEY_TEST5,_API_KEY_TEST6]

    for zticker in list (stocklist.ticker) :
        entity = str(zticker)
        print ("Stock Ticker Name : "+entity)
        _API_KEY_TEST = ""
        zkey = 0
        step = ""
        df_OV = pd.DataFrame()
        ISQ_reader = pd.DataFrame()
        CFQ_reader = pd.DataFrame()
        BSQ_reader = pd.DataFrame()
        dict_table = pd.DataFrame()

        randomize()

        #call data API Fundamental Quarterly
        fd = FundamentalData(key=_API_KEY_TEST, output_format='pandas')
        procedure = [step_1,step_2,step_3,step_4,step_5]

        while True :
            try : 
                for i in procedure :
                    i()

                #read csv file to pull data for calculation variable
                with open(r'C:\Varians\Income Statement Quarterly.csv') as ISQ:
                    ISQ_reader = pd.read_csv(ISQ)

                with open(r'C:\Varians\Cash Flow Quarterly.csv') as CFQ:
                    CFQ_reader = pd.read_csv(CFQ)

                with open(r'C:\Varians\Balance Sheet Quarterly.csv') as BSQ:
                    BSQ_reader = pd.read_csv(BSQ)

                with open(r'C:\Varians\Last Price.csv') as LP:
                    LP_reader = pd.read_csv(LP)

                #change date from object to datetime64
                ISQ_reader.fiscalDateEnding=ISQ_reader.fiscalDateEnding.astype('datetime64')
                ISQ_reader.replace(to_replace=['None'], value=np.nan, inplace=True) #to change string none to NaN
                ISQ_reader.fillna(value=0,inplace=True)
                CFQ_reader.fiscalDateEnding=CFQ_reader.fiscalDateEnding.astype('datetime64')
                CFQ_reader.replace(to_replace=['None'], value=np.nan, inplace=True)
                CFQ_reader.fillna(value=0,inplace=True)
                BSQ_reader.fiscalDateEnding=BSQ_reader.fiscalDateEnding.astype('datetime64')
                BSQ_reader.replace(to_replace=['None'], value=np.nan, inplace=True)
                BSQ_reader.fillna(value=0,inplace=True)
                BSQ_reader["commonStockSharesOutstanding"] = BSQ_reader.commonStockSharesOutstanding.astype(float) #to change  data type of SO
                LP_reader.date=LP_reader.date.astype('datetime64')

                cols = BSQ_reader.columns.drop('fiscalDateEnding')
                BSQ_reader[cols] = BSQ_reader[cols].apply(pd.to_numeric, errors='coerce')
                cols = ISQ_reader.columns.drop('fiscalDateEnding')
                ISQ_reader[cols] = ISQ_reader[cols].apply(pd.to_numeric, errors='coerce')
                cols = CFQ_reader.columns.drop('fiscalDateEnding')
                CFQ_reader[cols] = CFQ_reader[cols].apply(pd.to_numeric, errors='coerce')

                #make fundamental data from quarterly to monthly and fill forward NaN data with previous quarterly data
                #this code only apply on int or float data types
                #resample made the date ascending, while data from time series is descending. Why not just LP became ascending? Because LP has longer date than the other data
                ISQ_monthly=ISQ_reader.resample('M', on='fiscalDateEnding').mean()
                ISQ_monthly.fillna(method='ffill',inplace=True)
                ISQ_monthly.sort_values(by=['fiscalDateEnding'], ascending=False, inplace=True)
                CFQ_monthly=CFQ_reader.resample('M', on='fiscalDateEnding').mean()
                CFQ_monthly.fillna(method='ffill',inplace=True)
                CFQ_monthly.sort_values(by=['fiscalDateEnding'], ascending=False, inplace=True)
                BSQ_monthly=BSQ_reader.resample('M', on='fiscalDateEnding').mean()
                BSQ_monthly.fillna(method='ffill',inplace=True)
                BSQ_monthly.sort_values(by=['fiscalDateEnding'], ascending=False, inplace=True)

                #call all data needed for calculation and clean it first and wrap it into single variable list
                ISQ_monthly.reset_index(inplace=True)
                SO = BSQ_monthly['commonStockSharesOutstanding']
                netIncome = ISQ_monthly['netIncome']
                TA = BSQ_monthly['totalAssets']
                TL = BSQ_monthly['totalLiabilities']
                TSE = BSQ_monthly['totalShareholderEquity']
                cash = BSQ_monthly['cash']
                STD = BSQ_monthly['shortTermDebt']
                LTD = BSQ_monthly['longTermDebt']
                EBIT = ISQ_monthly['ebit']
                DEPR = CFQ_monthly['depreciation']
                AMOR = BSQ_monthly['accumulatedAmortization']

                dict_table=pd.DataFrame(list(zip(SO,netIncome,TA,TL,TSE,cash,STD,LTD,EBIT,DEPR,AMOR)))
                TS_table=LP_reader.loc[0:(len(netIncome)), ['date', '4. close']]

                tempaddrow=addrow(TS_table.date,ISQ_monthly.fiscalDateEnding,dict_table) #use addrow function to add row to equal TS table row with quarter table row
                datincome=pd.DataFrame(tempaddrow[0])
                counter=tempaddrow[1]


                allsindex=datincome.rename(columns={ 0 : 'SO', 1 : 'netIncome', 2 : 'TA', 3 : 'TL', 4 : 'TSE', 5 : 'cash', 6 : 'STD', 7 : 'LTD', 8 : 'EBIT', 9 : 'DEPR', 10 : 'AMOR' })
                TS_table=LP_reader.loc[0:(len(netIncome)+counter), ['date', '4. close']]
                TS_table=TS_table.rename(columns={"4. close": "LastPrice"})
                inputmerge=TS_table.merge(allsindex, left_index=True, right_index=True)
                #print(inputmerge)
                #print(tempaddrow)

                #calculating EPS TTM (Trailing Twelve Months) for PER input

                #calculation for EPS Quarterly

                Tableeps =pd.DataFrame()
                Tableeps ['Date'] = ISQ_reader.fiscalDateEnding
                Tableeps ['NIQ'] = ISQ_reader.netIncome
                Tableeps ['SOQ'] = BSQ_reader.commonStockSharesOutstanding
                Tableeps = Tableeps.sort_index(ascending=False)
                Tableeps = Tableeps.reset_index(drop=True)
                EPStemps = [Tableeps.NIQ[i]/Tableeps.SOQ[i] for i in range(len(Tableeps.SOQ))]
                NIQtemps = Tableeps.NIQ

                #Make EPSTTM table from function makettm 
                EPSTTM = makettm(EPStemps,4)
                #EPSTTM = EPSTTM.rename(columns={'EPS':'EPSTTM'})
                Tableeps ['EPSTTM'] = pd.DataFrame(EPSTTM)

                #make netIncome TTM table from function makettm
                NITTM = makettm(NIQtemps,4)
                Tableeps['NITTM']=pd.DataFrame(NITTM)

                Tableeps=Tableeps.resample('M', on='Date').mean()
                Tableeps.fillna(method='ffill',inplace=True)
                Tableeps = Tableeps.sort_index(ascending=False)
                Tableeps.reset_index(inplace=True)
                Coldate = Tableeps.Date
                Tableeps = Tableeps.set_index('Date')
                temp= addrow(TS_table.date,Coldate,Tableeps)
                TTMmonth = temp[0]
                #print(TTMmonth)
                #print(Tableeps)
                #gimana ngeluarin function result ke dataframe

                #calculation
                #columns=['EPS']

                Calculation = pd.DataFrame()

                inputcalc = inputmerge
                inputcalc = inputcalc.sort_index(ascending=False)
                inputcalc = inputcalc.reset_index(drop=True)

                #for i in range (len(SO)):
                Calculation ['Date'] = inputcalc.date
                Calculation ['LastPrice'] = inputcalc.LastPrice
                Calculation ['ShareOut'] = inputcalc.SO
                Calculation ['MarketCap'] = [inputcalc.LastPrice[i]*inputcalc.SO[i] for i in range(len(inputcalc.SO))]
                Calculation ['EPS'] = [inputcalc.netIncome[i]/inputcalc.SO[i] for i in range(len(inputcalc.SO))]

                #Merging TTM EPS
                temp = pd.DataFrame()
                temp['EPSTTM'] = (TTMmonth.EPSTTM)
                temp['NITTM'] = TTMmonth.NITTM
                temp = temp.sort_index(ascending=False)
                temp.reset_index(inplace=True)
                #Calculation = Calculation.merge(EPSTTM, left_index=True, right_index=True)

                Calculation ['EPSTTM'] = temp.EPSTTM
                Calculation ['PER'] = [inputcalc.LastPrice[i]/temp.EPSTTM[i] for i in range(len(inputcalc.SO))]
                Calculation ['BVPS'] = [(inputcalc.TA[i]-inputcalc.TL[i])/inputcalc.SO[i] for i in range(len(inputcalc.SO))]
                Calculation ['PBV'] = [Calculation.LastPrice[i]/Calculation.BVPS[i] for i in range(len(inputcalc.SO))]
                Calculation ['ROA%'] = [temp.NITTM[i]/inputcalc.TA[i]*100 for i in range(len(inputcalc.SO))]
                Calculation ['ROE%'] = [temp.NITTM[i]/inputcalc.TSE[i]*100 for i in range(len(inputcalc.SO))]
                Calculation ['EV'] = [Calculation.MarketCap[i]+inputcalc.cash[i]-(inputcalc.STD[i]+inputcalc.LTD[i]) for i in range(len(inputcalc.SO))]
                Calculation ['EBITDA'] = [inputcalc.EBIT[i]+inputcalc.DEPR[i]+inputcalc.AMOR[i] for i in range(len(inputcalc.SO))]
                Calculation ['EV/EBITDA'] = [Calculation.EV[i]/Calculation.EBITDA[i] for i in range(len(inputcalc.SO))]
                Calculation ['D/E'] = [inputcalc.TL[i]/inputcalc.TSE[i] for i in range(len(inputcalc.TL))]
                Calculation ['Debt/Totalcap'] = [(inputcalc.STD[i]+inputcalc.LTD[i])/(inputcalc.STD[i]+inputcalc.LTD[i]+inputcalc.TSE[i]) for i in range(len(inputcalc.STD))]
                Calculation ['Debt/EBITDA'] = [(inputcalc.STD[i]+inputcalc.LTD[i])/Calculation.EBITDA[i] for i in range(len(inputcalc.STD)) ]

                Calculation = Calculation.replace([np.inf, -np.inf], 0)
                #print(Calculation.tail(30))
                print(Calculation.dtypes)

                Calculation.to_csv(r'C:\Varians\Calculation.csv', index=None, header=True)

                #preparing pandas dataframe to list inputing to mysql

                mysql_list = []
                Unique_ID  = []
                StockName  = [entity.upper()]*len(Calculation['Date'])  

                for i in range (len(Calculation['Date'])) :
                    temp = entity + str(Calculation['Date'][i].strftime('%Y%m%d'))
                    Unique_ID.append(temp)
                    i += 1

                mysql_list.append(Unique_ID)
                mysql_list.append(StockName)

                for i in list(Calculation.columns) :
                    mysql_list.append(Calculation[i])

                #print(mysql_list)
                # data table overview

                df_OV = pd.Series(OV, name='Info')
                df_OV.index.name = 'Data'
                list_header_OV = df_OV.index.tolist()
                #print (df_OV.head(5))

                db = connect("newstockmarket")
                cursor = db.cursor()

                #--------------------------------------------------------------------------------------------------------------------------------------------------
                ftable = "stock_fundamental"
                ovtable = "stock_overview"
                fintable = "financial_report"

                create_ovtable = ("CREATE TABLE IF NOT EXISTS {table} "
                                  #" ( Stock_ID int(5) NOT NULL AUTO_INCREMENT PRIMARY KEY, "
                                  " ( SYMBOL VARCHAR(8) NOT NULL PRIMARY KEY, ASSET_TYPE VARCHAR(30), NAME VARCHAR(50) NOT NULL, "
                                  " DESCRIPTION VARCHAR(255), EXCHANGE VARCHAR(10), CURRENCY VARCHAR(5), "
                                  " COUNTRY VARCHAR(50), SECTOR VARCHAR(50), INDUSTRY VARCHAR(50), "
                                  " ADDRESS VARCHAR(255), Full_Time_Employees INT(20), Fiscal_Year_End VARCHAR(15), "
                                  " LATEST_QUARTER DATE, DividendDate DATE, ExDividendDate DATE, "
                                  " LastSplitFactor DECIMAL(5,2), LastSplitDate DATE )"
                                 )
                create_fintable = ("CREATE TABLE IF NOT EXISTS {table} "
                                  " ( NUMBER int(5) NOT NULL AUTO_INCREMENT PRIMARY KEY, "
                                  " SYMBOL VARCHAR(8), MARKET_CAP DECIMAL(15,2), EBITDA DECIMAL(15,2), "
                                  " PER DECIMAL(15,2), PEGR DECIMAL(15,2), BOOK_VALUE DECIMAL(15,2), "
                                  " Dividend_Per_Share DECIMAL(15,2), Dividend_Yield DECIMAL(15,2), EPS DECIMAL(15,2), "
                                  " Revenue_Per_Share_TTM DECIMAL(15,2), Profit_Margin DECIMAL(15,2), Operating_Margin_TTM DECIMAL(15,2), "
                                  " ROA_TTM DECIMAL(15,2), ROE_TTM DECIMAL(15,2), REVENUE_TTM DECIMAL(15,2), "
                                  " Gross_Profit_TTM DECIMAL(15,2), Diluted_EPS_TTM DECIMAL(15,2), Quarterly_Earnings_Growth_YOY DECIMAL(15,2), "
                                  " Quarterly_Revenue_Growth_YOY DECIMAL(15,2), Analyst_Target_Price DECIMAL(15,2), Trailing_PE DECIMAL(15,2), "
                                  " Forward_PE DECIMAL(15,2), Price_to_Sales_Ratio_TTM DECIMAL(15,2), PBV DECIMAL(15,2), "
                                  " EVtoRevenue DECIMAL(15,2), EVtoEBITDA DECIMAL(15,2), Beta DECIMAL(15,2), "
                                  " 52WeekHigh DECIMAL(15,2), 52WeekLow DECIMAL(15,2), 50DayMovingAverage DECIMAL(15,2), "
                                  " 200DayMovingAverage DECIMAL(15,2), SharesOutstanding DECIMAL(15,2), SharesFloat DECIMAL(15,2), " 
                                  " SharesShort DECIMAL(15,2), SharesShortPriorMonth DECIMAL(15,2), ShortRatio DECIMAL(15,2), " 
                                  " ShortPercentOutstanding DECIMAL(15,2), ShortPercentFloat DECIMAL(15,2), PercentInsiders DECIMAL(15,2), "
                                  " PercentInstitutions DECIMAL(15,2), ForwardAnnualDividendRate DECIMAL(15,2), ForwardAnnualDividendYield DECIMAL(15,2), " 
                                  " PayoutRatio DECIMAL(15,2) ) "
                                  )
                create_ftable = ("CREATE TABLE IF NOT EXISTS {table} "
                               " ( UNIQUE_ID VARCHAR(20) NOT NULL PRIMARY KEY, SYMBOL VARCHAR(8) NOT NULL, "
                               " DATE date NOT NULL, LAST_PRICE_RP decimal(8, 2) NOT NULL,"
                               " SHARE_OUT decimal(15, 2) NOT NULL, MARKET_CAP_RP decimal(15, 2) NOT NULL, "
                               " DEVIDEN_RP decimal(15, 2) NOT NULL,  EPSTTMM_RP decimal(15, 2) NOT NULL, "
                               " PER_X decimal(15, 2) NOT NULL, BVPS_RP decimal(15, 2) NOT NULL, "
                               " PBV_X decimal(15, 2) NOT NULL, ROA_PERCENT decimal(15, 2) NOT NULL, "
                               " ROE_PERCENT decimal(15, 2) NOT NULL, EV decimal(15, 2) NOT NULL, "
                               " EBITDA decimal(15, 2) NOT NULL, EV_EBITDA_RATIO decimal(15, 2) NOT NULL, "
                               " D_E_RATIO decimal(15, 2) NOT NULL, DEBT_TOTALCAP_RATIO decimal(15, 2) NOT NULL, "
                               " DEBT_EBITDA_RATIO decimal(15, 2) NOT NULL) "
                               )

                cursor = db.cursor()
                cursor.execute(create_ovtable.format(table=ovtable) )                    
                cursor.execute(create_fintable.format(table=fintable, atable=fintable) )                
                cursor.execute(create_ftable.format(table=ftable, atable=ftable) )

                #------------------------------------------------------------------------------------------
                insert_ovtable = ("INSERT IGNORE INTO {table} "
                                "( SYMBOL, ASSET_TYPE, "
                                "NAME, DESCRIPTION, EXCHANGE, "
                                "CURRENCY, COUNTRY, SECTOR, "
                                "INDUSTRY, ADDRESS, Full_Time_Employees, "
                                "Fiscal_Year_End, LATEST_QUARTER, DividendDate, "
                                "ExDividendDate, LastSplitFactor, LastSplitDate ) "
                                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                                 )
                temp_data = []

                for i in range(13) :
                    single_data = df_OV[list_header_OV[i]]
                    temp_data.append(single_data)
                for i in range(55, len(list_header_OV)) :
                    single_data = df_OV[list_header_OV[i]]
                    temp_data.append(single_data)
                cursor.execute(insert_ovtable.format(table=ovtable), temp_data)

                insert_fintable = ("INSERT IGNORE INTO {table} "
                                    "( SYMBOL, MARKET_CAP, EBITDA, " 
                                    " PER, PEGR, BOOK_VALUE, "
                                    " Dividend_Per_Share, Dividend_Yield, EPS, "
                                    " Revenue_Per_Share_TTM, Profit_Margin, Operating_Margin_TTM, "
                                    " ROA_TTM, ROE_TTM, REVENUE_TTM, "
                                    " Gross_Profit_TTM, Diluted_EPS_TTM, Quarterly_Earnings_Growth_YOY, "
                                    " Quarterly_Revenue_Growth_YOY, Analyst_Target_Price, Trailing_PE, "
                                    " Forward_PE, Price_to_Sales_Ratio_TTM, PBV, "
                                    " EVtoRevenue, EVtoEBITDA, Beta, "
                                    " 52WeekHigh, 52WeekLow, 50DayMovingAverage, "
                                    " 200DayMovingAverage, SharesOutstanding, SharesFloat, "
                                    " SharesShort, SharesShortPriorMonth, ShortRatio, "
                                    " ShortPercentOutstanding, ShortPercentFloat, PercentInsiders, "
                                    " PercentInstitutions, ForwardAnnualDividendRate, ForwardAnnualDividendYield, PayoutRatio ) "
                                    " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "
                                    " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "
                                    " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s )"
                                  )
                temp_data = []
                temp_data.append(entity)
                for i in range(13, 55) :
                    single_data = df_OV[list_header_OV[i]]
                    temp_data.append(single_data)
                cursor.execute(insert_fintable.format(table=fintable), temp_data)

                insert_ftable = ("INSERT IGNORE INTO {table} "
                               "(UNIQUE_ID, SYMBOL, DATE, LAST_PRICE_RP, SHARE_OUT, MARKET_CAP_RP, "
                               "DEVIDEN_RP, EPSTTMM_RP, PER_X, BVPS_RP, PBV_X, ROA_PERCENT, ROE_PERCENT, "
                               "EV, EBITDA, EV_EBITDA_RATIO, D_E_RATIO, DEBT_TOTALCAP_RATIO, "
                               "DEBT_EBITDA_RATIO) "
                               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
                j = 0
                while j < len(mysql_list[0]) :
                    temp_data = []
                    i = 0
                    while i < len(mysql_list) :
                        temp_data.append(str (mysql_list[i][j]))
                        #print(project_data)
                        i += 1

                    cursor.execute(insert_ftable.format(table=ftable), temp_data)
                    j += 1

                db.commit()
                db.close()
                time.sleep(10)
                break
            except ValueError as e :
                print (e)
                if str(e) == "Error getting data from the api, no return was given." :
                    print ("Problem Ticker Name : "+entity+" ...Skipping")
                    break
                    continue

                else :
                    if (step == "step_2") :
                        procedure = [step_2,step_3,step_4,step_5]
                        print ("beginning from step 2")
                        randomize()
                        time.sleep(10)  
                    elif (step == "step_3") :
                        procedure = [step_3,step_4,step_5]
                        print ("beginning from step 3")
                        randomize()
                        time.sleep(10)  
                    elif (step == "step_4") :
                        procedure = [step_4,step_5]
                        print ("beginning from step 4")
                        randomize()
                        time.sleep(10) 
                    elif (step == "step_5") :
                        procedure = [step_5]
                        print ("beginning from step 5")
                        randomize()
                        time.sleep(10)  
                    else :
                        print ("Step 1 Problem API Limit..Try Again")
                        print (step)
                        randomize()
                        time.sleep(10)        
#     except ValueError :
#         time.sleep(5)
#         continue
        
#######################################################################################################################################################################################################################################################################################################

Creation of the directory /Varians failed
Stock Ticker Name : DPSGY
the API key right now is : 6PE06AMZAM1MCGFX
Error getting data from the api, no return was given.
Problem Ticker Name : DPSGY ...Skipping
Stock Ticker Name : UPS
the API key right now is : 6PE06AMZAM1MCGFX
Succsesfully Pull Data 1 from web
Succsesfully Pull Data 2 from web
Succsesfully Pull Data 3 from web
Succsesfully Pull Data Overview from web
Succsesfully Pull Data Timeseries from web
Date             datetime64[ns]
LastPrice               float64
ShareOut                float64
MarketCap               float64
EPS                     float64
EPSTTM                  float64
PER                     float64
BVPS                    float64
PBV                     float64
ROA%                    float64
ROE%                    float64
EV                      float64
EBITDA                  float64
EV/EBITDA               float64
D/E                     float64
Debt/Totalcap           float64
Debt/EBITDA             floa

<ipython-input-4-81e8ac78b0cf>:236: RuntimeWarning: divide by zero encountered in true_divide
  EPStemps = [Tableeps.NIQ[i]/Tableeps.SOQ[i] for i in range(len(Tableeps.SOQ))]
<ipython-input-4-81e8ac78b0cf>:274: RuntimeWarning: divide by zero encountered in double_scalars
  Calculation ['EPS'] = [inputcalc.netIncome[i]/inputcalc.SO[i] for i in range(len(inputcalc.SO))]
<ipython-input-4-81e8ac78b0cf>:285: RuntimeWarning: divide by zero encountered in double_scalars
  Calculation ['PER'] = [inputcalc.LastPrice[i]/temp.EPSTTM[i] for i in range(len(inputcalc.SO))]
<ipython-input-4-81e8ac78b0cf>:286: RuntimeWarning: divide by zero encountered in double_scalars
  Calculation ['BVPS'] = [(inputcalc.TA[i]-inputcalc.TL[i])/inputcalc.SO[i] for i in range(len(inputcalc.SO))]


KeyboardInterrupt: 

In [5]:
ISQ_reader

,fiscalDateEnding,reportedCurrency,totalRevenue,totalOperatingExpense,costOfRevenue,grossProfit,ebit,netIncome,researchAndDevelopment,effectOfAccountingCharges,...,netInterestIncome,extraordinaryItems,nonRecurring,otherItems,incomeTaxExpense,totalOtherIncomeExpense,discontinuedOperations,netIncomeFromContinuingOperations,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments
0,2020-12-31,NaN,24896000000,16011000000,6848000000,18048000000,-4113000000,-3347000000,0,0,...,-175000000,0,0,0,-941000000,-6150000000,0,-3347000000,-3347000000,0.0
1,2020-09-30,NaN,21238000000,13368000000,17261000000,3977000000,3040000000,1957000000,0,0,...,-176000000,0,0,0,568000000,338000000,0,1957000000,1957000000,0.0
2,2020-06-30,NaN,20459000000,13123000000,16628000000,3831000000,2873000000,1768000000,0,0,...,-183000000,0,0,0,589000000,328000000,0,1768000000,1768000000,0.0
3,2020-03-31,NaN,18035000000,12325000000,15372000000,2663000000,1720000000,965000000,0,0,...,-167000000,0,0,0,285000000,345000000,0,965000000,965000000,0.0
4,2019-12-31,NaN,20568000000,13158000000,16609000000,3959000000,2763000000,-106000000,0,0,...,-166000000,0,0,0,-92000000,-2165000000,0,-106000000,-106000000,0.0
5,2019-09-30,NaN,18318000000,11551000000,14816000000,3502000000,2715000000,1750000000,0,0,...,-159000000,0,0,0,456000000,237000000,0,1750000000,1750000000,0.0
6,2019-06-30,NaN,18048000000,11253000000,14526000000,3522000000,2718000000,1685000000,0,0,...,-159000000,0,0,0,519000000,220000000,0,1685000000,1685000000,0.0
7,2019-03-31,NaN,17160000000,11225000000,14426000000,2734000000,1962000000,1111000000,0,0,...,-169000000,0,0,0,329000000,215000000,0,1111000000,1111000000,0.0
8,2018-12-31,NaN,19848000000,12250000000,16290000000,3558000000,2549000000,453000000,0,0,...,0,0,0,0,90000000,1318000000,0,453000000,453000000,0.0
9,2018-09-30,NaN,17444000000,10876000000,14380000000,3064000000,2251000000,1508000000,0,0,...,0,0,0,0,381000000,-219000000,0,1508000000,1508000000,0.0


In [6]:
CFQ_reader


,fiscalDateEnding,reportedCurrency,investments,changeInLiabilities,cashflowFromInvestment,otherCashflowFromInvestment,netBorrowings,cashflowFromFinancing,otherCashflowFromFinancing,changeInOperatingActivities,...,depreciation,dividendPayout,stockSaleAndPurchase,changeInInventory,changeInAccountReceivables,changeInNetIncome,capitalExpenditures,changeInReceivables,changeInExchangeRate,changeInCashAndCashEquivalents
0,2020-12-31,NaN,19000000,1540000000,-2177000000,-3000000,-1184000000,-1961000000,-2000000,-102000000,...,712000000,-846000000,71000000,0,-1914000000,4443000000,2193000000,-1914000000,0,-2962000000
1,2020-09-30,NaN,5000000,702000000,-1163000000,-14000000,-1385000000,-2157000000,-17000000,530000000,...,677000000,-845000000,90000000,0,247000000,-395000000,1154000000,247000000,0,16000000
2,2020-06-30,NaN,123000000,1478000000,-1009000000,-1000000,-1757000000,-2566000000,-16000000,917000000,...,661000000,-843000000,50000000,0,-1118000000,519000000,1132000000,-1118000000,0,-178000000
3,2020-03-31,NaN,3000000,-792000000,-934000000,-4000000,3475000000,2167000000,-318000000,394000000,...,648000000,-840000000,-150000000,0,1223000000,288000000,933000000,1223000000,0,3783000000
4,2019-12-31,NaN,-3000000,1178000000,-2034000000,13000000,1128000000,127000000,-8000000,-231000000,...,630000000,-797000000,-196000000,0,-1560000000,2818000000,2044000000,-1560000000,0,1039000000
5,2019-09-30,NaN,233000000,-129000000,-1180000000,62000000,677000000,-320000000,17000000,466000000,...,587000000,-798000000,-216000000,0,26000000,-1114000000,1475000000,26000000,0,-14000000
6,2019-06-30,NaN,-6000000,145000000,-1431000000,-78000000,129000000,-839000000,7000000,78000000,...,575000000,-800000000,-175000000,0,-484000000,-236000000,1347000000,-484000000,0,-340000000
7,2019-03-31,NaN,111000000,-1224000000,-1416000000,-13000000,485000000,-695000000,-182000000,121000000,...,568000000,-799000000,-199000000,0,1301000000,400000000,1514000000,1301000000,0,166000000
8,2018-12-31,NaN,-54000000,1906000000,-1831000000,16000000,-475000000,-1432000000,-29000000,-105000000,...,545000000,-751000000,-177000000,0,-1517000000,2449000000,1793000000,-1517000000,0,26000000
9,2018-09-30,NaN,-32000000,30000000,-1679000000,-6000000,286000000,-653000000,12000000,176000000,...,524000000,-753000000,-198000000,0,-174000000,376000000,1641000000,-174000000,0,-110000000


In [7]:
BSQ_reader

,fiscalDateEnding,reportedCurrency,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiabilities,totalShareholderEquity,deferredLongTermLiabilities,otherCurrentLiabilities,...,otherNonCurrentLiabilities,totalNonCurrentLiabilities,negativeGoodwill,warrants,preferredStockRedeemable,capitalSurplus,liabilitiesAndShareholderEquity,cashAndShortTermInvestments,accumulatedDepreciation,commonStockSharesOutstanding
0,2020-12-31,NaN,62408000000,2274000000,0.0,5951000000,59199000000,657000000,488000000,3284000000,...,19664000000,42183000000,0.0,0.0,0.0,865000000,62408000000,6316000000,0.0,0.0
1,2020-09-30,NaN,62406000000,2289000000,0.0,1696000000,54327000000,5592000000,2145000000,3197000000,...,13390000000,38871000000,0.0,0.0,0.0,490000000,62406000000,9241000000,0.0,0.0
2,2020-06-30,NaN,62057000000,2207000000,0.0,1780000000,55218000000,4363000000,1918000000,3249000000,...,13830000000,38947000000,0.0,0.0,0.0,255000000,62057000000,9216000000,0.0,0.0
3,2020-03-31,NaN,60895000000,2176000000,0.0,1803000000,55089000000,3299000000,1787000000,3076000000,...,13260000000,39243000000,0.0,0.0,0.0,29000000,60895000000,9460000000,0.0,869000000.0
4,2019-12-31,NaN,57857000000,2167000000,0.0,1428000000,52183000000,3267000000,1632000000,2972000000,...,13320000000,36770000000,0.0,0.0,0.0,150000000,57857000000,5741000000,0.0,869000000.0
5,2019-09-30,NaN,53282000000,2131000000,0.0,1491000000,45645000000,5558000000,1886000000,2504000000,...,9184000000,32810000000,0.0,0.0,0.0,129000000,53282000000,4538000000,0.0,870000000.0
6,2019-06-30,NaN,52787000000,2121000000,0.0,1374000000,46338000000,4405000000,1947000000,2857000000,...,10331000000,32705000000,0.0,0.0,0.0,102000000,52787000000,4797000000,0.0,865000000.0
7,2019-03-31,NaN,52061000000,2093000000,0.0,1333000000,46483000000,3476000000,1758000000,2817000000,...,11139000000,33274000000,0.0,0.0,0.0,27000000,52061000000,5111000000,0.0,869000000.0
8,2018-12-31,NaN,50016000000,2075000000,0.0,1277000000,46979000000,3021000000,1619000000,2237000000,...,11342000000,32892000000,0.0,0.0,0.0,0,50016000000,5035000000,0.0,870000000.0
9,2018-09-30,NaN,46689000000,2070000000,0.0,1336000000,43563000000,3107000000,1069000000,1888000000,...,10171000000,31341000000,0.0,0.0,0.0,0,46689000000,4841000000,0.0,870000000.0


In [9]:
print(mysql_list)

[['UPS20151231', 'UPS20160129', 'UPS20160229', 'UPS20160331', 'UPS20160429', 'UPS20160531', 'UPS20160630', 'UPS20160729', 'UPS20160831', 'UPS20160930', 'UPS20161031', 'UPS20161130', 'UPS20161230', 'UPS20170131', 'UPS20170228', 'UPS20170331', 'UPS20170428', 'UPS20170531', 'UPS20170630', 'UPS20170731', 'UPS20170831', 'UPS20170929', 'UPS20171031', 'UPS20171130', 'UPS20171229', 'UPS20180131', 'UPS20180228', 'UPS20180329', 'UPS20180430', 'UPS20180531', 'UPS20180629', 'UPS20180731', 'UPS20180831', 'UPS20180928', 'UPS20181031', 'UPS20181130', 'UPS20181231', 'UPS20190131', 'UPS20190228', 'UPS20190329', 'UPS20190430', 'UPS20190531', 'UPS20190628', 'UPS20190731', 'UPS20190830', 'UPS20190930', 'UPS20191031', 'UPS20191129', 'UPS20191231', 'UPS20200131', 'UPS20200228', 'UPS20200331', 'UPS20200430', 'UPS20200529', 'UPS20200630', 'UPS20200731', 'UPS20200831', 'UPS20200930', 'UPS20201030', 'UPS20201130', 'UPS20201231', 'UPS20210129', 'UPS20210224'], ['UPS', 'UPS', 'UPS', 'UPS', 'UPS', 'UPS', 'UPS', 'U